In [1]:
# to make this notebook's output stable across runs
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"  # silent warning for TF 0 = all logs, 1 = INFO, 2 = WARNING, 3 = ERROR 

# set up TensorFlow
import tensorflow as tf

print("TensorFlow version:", tf.__version__)

TensorFlow version: 2.20.0


In [2]:
from  IPython import display
from matplotlib import pyplot as plt

import numpy as np

In [3]:
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()

train_labels = train_labels[:1000]
test_labels = test_labels[:1000]

train_images = train_images[:1000].reshape(-1, 28 * 28) / 255.0
test_images = test_images[:1000].reshape(-1, 28 * 28) / 255.0

In [4]:
# Define a model --  Define a simple sequential model
def create_model():
    model = tf.keras.Sequential([
        tf.keras.layers.Input(shape=(784,)),   # consistent with data
        tf.keras.layers.Dense(512, activation='relu'),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(10)               # logits
    ])

    model.compile(
        optimizer='adam',
        loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
        metrics=[tf.keras.metrics.SparseCategoricalAccuracy()]
    )

    return model

model = create_model()
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 512)            │       401,920 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 10)             │         5,130 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 407,050 (1.55 MB)

 Trainable params: 407,050 (1.55 MB)

 Non-trainable params: 0 (0.00 B)

In [5]:
checkpoint_path1 = "tensorflow_training_1/cp.weights.h5"
checkpoint_dir1 = os.path.dirname(checkpoint_path1)

# Create a callback that saves the model's weights
cp_callback1 = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path1,
                                                 save_weights_only=True,
                                                 verbose=1)

# Train the model with the new callback
model.fit(train_images, 
          train_labels,  
          epochs=10,
          validation_data=(test_images, test_labels),
          callbacks=[cp_callback1])  # Pass callback to training

Epoch 1/10
31/32 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 1.6303 - sparse_categorical_accuracy: 0.4805
Epoch 1: saving model to tensorflow_training_1/cp.weights.h5
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - loss: 1.1330 - sparse_categorical_accuracy: 0.6650 - val_loss: 0.6949 - val_sparse_categorical_accuracy: 0.7860
Epoch 2/10
23/32 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.4288 - sparse_categorical_accuracy: 0.8723
Epoch 2: saving model to tensorflow_training_1/cp.weights.h5
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.4168 - sparse_categorical_accuracy: 0.8860 - val_loss: 0.5208 - val_sparse_categorical_accuracy: 0.8340
Epoch 3/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.2960 - sparse_categorical_accuracy: 0.9242
Epoch 3: saving model to tensorflow_training_1/cp.weights.h5
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.2821 - sparse_categorical_accuracy: 0.9270 - val_loss: 0.4775 - val_sparse_categorical_accuracy: 0.8460
Epoch 4/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/st

In [6]:
# This creates a single collection of TensorFlow checkpoint files that are updated at the end of each epoch:
os.listdir(checkpoint_dir1)

['cp.weights.h5']

In [7]:
# Create a basic model instance
model1 = create_model()

# Evaluate the model
loss, acc = model1.evaluate(test_images, test_labels, verbose=2)
print("Untrained model, accuracy: {:5.2f}%".format(100 * acc))

32/32 - 0s - 5ms/step - loss: 2.3099 - sparse_categorical_accuracy: 0.2080
Untrained model, accuracy: 20.80%


In [25]:
# Loads the weights
model1.load_weights(checkpoint_path1)

model1.compile(
    optimizer='adam',
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy']
)

# Re-evaluate the model
loss, acc = model1.evaluate(test_images, test_labels, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100 * acc))

32/32 - 0s - 7ms/step - accuracy: 0.8690 - loss: 0.4097
Restored model, accuracy: 86.90%


In [26]:
# Include the epoch in the file name (uses `str.format`)
checkpoint_path2 = "tensorflow_training_2/cp-{epoch:04d}.weights.h5"
checkpoint_dir2 = os.path.dirname(checkpoint_path2)

batch_size = 32

# Calculate the number of batches per epoch
import math
n_batches = len(train_images) / batch_size
n_batches = math.ceil(n_batches)    # round up the number of batches to the nearest whole integer

# Create a callback that saves the model's weights every 5 epochs
cp_callback2 = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path2, 
    verbose=1, 
    save_weights_only=True,
    save_freq=5*n_batches)

# Create a new model instance
model2 = create_model()

# Save the weights using the `checkpoint_path` format
model2.save_weights(checkpoint_path2.format(epoch=0))

# Train the model with the new callback
model2.fit(train_images, 
          train_labels,
          epochs=50, 
          batch_size=batch_size, 
          callbacks=[cp_callback2],
          validation_data=(test_images, test_labels),
          verbose=0)


Epoch 5: saving model to tensorflow_training_2/cp-0005.weights.h5

Epoch 10: saving model to tensorflow_training_2/cp-0010.weights.h5

Epoch 15: saving model to tensorflow_training_2/cp-0015.weights.h5

Epoch 20: saving model to tensorflow_training_2/cp-0020.weights.h5

Epoch 25: saving model to tensorflow_training_2/cp-0025.weights.h5

Epoch 30: saving model to tensorflow_training_2/cp-0030.weights.h5

Epoch 35: saving model to tensorflow_training_2/cp-0035.weights.h5

Epoch 40: saving model to tensorflow_training_2/cp-0040.weights.h5

Epoch 45: saving model to tensorflow_training_2/cp-0045.weights.h5

Epoch 50: saving model to tensorflow_training_2/cp-0050.weights.h5


In [27]:
os.listdir(checkpoint_dir2)

['cp-0045.weights.h5',
 'cp-0010.weights.h5',
 'cp-0000.weights.h5',
 'cp-0025.weights.h5',
 'cp-0020.weights.h5',
 'cp-0030.weights.h5',
 'cp-0035.weights.h5',
 'cp-0005.weights.h5',
 'cp-0040.weights.h5',
 'cp-0015.weights.h5',
 'cp-0050.weights.h5']

In [28]:
latest = tf.train.latest_checkpoint(checkpoint_dir2)
latest

In [29]:
latest = 'tensorflow_training_2/cp-0050.weights.h5'

In [30]:
# Create a new model instance
model2 = create_model()

# Load the previously saved weights
model2.load_weights(latest)

# Re-evaluate the model
loss, acc = model2.evaluate(test_images, test_labels, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100 * acc))

32/32 - 0s - 5ms/step - loss: 0.4984 - sparse_categorical_accuracy: 0.8730
Restored model, accuracy: 87.30%


In [31]:
# Save the weights
model2.save_weights('./tensorflow_training_2/cp-0050.weights.h5')

# Create a new model instance
model2 = create_model()

# Restore the weights
model2.load_weights('./tensorflow_training_2/cp-0050.weights.h5')

# Evaluate the model
loss, acc = model2.evaluate(test_images, test_labels, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100 * acc))

32/32 - 0s - 5ms/step - loss: 0.4984 - sparse_categorical_accuracy: 0.8730
Restored model, accuracy: 87.30%


In [32]:
# Create and train a new model instance.
model3 = create_model()
model3.fit(train_images, train_labels, epochs=5)

# Save the entire model as a `.keras` zip archive.
model3.save('saved_models/my_model.keras')

Epoch 1/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 1.1585 - sparse_categorical_accuracy: 0.6840
Epoch 2/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.4146 - sparse_categorical_accuracy: 0.8770
Epoch 3/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.2773 - sparse_categorical_accuracy: 0.9220
Epoch 4/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.2196 - sparse_categorical_accuracy: 0.9450
Epoch 5/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.1567 - sparse_categorical_accuracy: 0.9650


In [33]:
new_model = tf.keras.models.load_model('saved_models/my_model.keras')

# Show the model architecture
new_model.summary()

Model: "sequential_11"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_24 (Dense)                │ (None, 512)            │       401,920 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_11 (Dropout)            │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_25 (Dense)                │ (None, 10)             │         5,130 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,221,152 (4.66 MB)

 Trainable params: 407,050 (1.55 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 814,102 (3.11 MB)

In [34]:
# Evaluate the restored model
loss, acc = new_model.evaluate(test_images, test_labels, verbose=2)
print('Restored model, accuracy: {:5.2f}%'.format(100 * acc))

print(new_model.predict(test_images).shape)

32/32 - 0s - 4ms/step - loss: 0.4217 - sparse_categorical_accuracy: 0.8600
Restored model, accuracy: 86.00%
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
(1000, 10)


In [35]:
# Create and train a new model instance.
model4 = create_model()
model4.fit(train_images, train_labels, epochs=5)

# Save the entire model as a SavedModel.
!mkdir -p saved_models
model.save('saved_models/tensorflow_learning_model.keras')

Epoch 1/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 1.1771 - sparse_categorical_accuracy: 0.6590
Epoch 2/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.4391 - sparse_categorical_accuracy: 0.8750
Epoch 3/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.2879 - sparse_categorical_accuracy: 0.9220
Epoch 4/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.2100 - sparse_categorical_accuracy: 0.9550
Epoch 5/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.1642 - sparse_categorical_accuracy: 0.9660


In [36]:
new_model = tf.keras.models.load_model('saved_models/tensorflow_learning_model.keras')

# Check its architecture
new_model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 512)            │       401,920 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 10)             │         5,130 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,221,152 (4.66 MB)

 Trainable params: 407,050 (1.55 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 814,102 (3.11 MB)

Introduction to the Keras Tuner

The Keras Tuner is a library that helps you pick the optimal set of hyperparameters for your TensorFlow program. The process of selecting the right set of hyperparameters for your machine learning (ML) application is called hyperparameter tuning or hypertuning.

Hyperparameters are the variables that govern the training process and the topology of an ML model. These variables remain constant over the training process and directly impact the performance of your ML program. Hyperparameters are of two types:

Model hyperparameters which influence model selection such as the number and width of hidden layers
Algorithm hyperparameters which influence the speed and quality of the learning algorithm such as the learning rate for Stochastic Gradient Descent (SGD) and the number of nearest neighbors for a k Nearest Neighbors (KNN) classifier

In [37]:
import tensorflow as tf
from tensorflow import keras
import keras_tuner as kt

In [38]:
# Download the dataset
(img_train, label_train), (img_test, label_test) = keras.datasets.fashion_mnist.load_data()

In [39]:
# Normalize pixel values between 0 and 1
img_train = img_train.astype('float32') / 255.0
img_test = img_test.astype('float32') / 255.0

In [40]:
# Define the model
def model_builder(hp):
  model = tf.keras.Sequential([
          tf.keras.Input(shape=(28, 28)),
          tf.keras.layers.Flatten(),
          tf.keras.layers.Dense(128, activation='relu'),
          tf.keras.layers.Dropout(0.2),
          tf.keras.layers.Dense(10)
        ])

  # Tune the number of units in the first Dense layer
  # Choose an optimal value between 32-512
  hp_units = hp.Int('units', min_value=32, max_value=512, step=32)
  model.add(keras.layers.Dense(units=hp_units, activation='relu'))
  model.add(keras.layers.Dense(10))

  # Tune the learning rate for the optimizer
  # Choose an optimal value from 0.01, 0.001, or 0.0001
  hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])

  model.compile(optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate),
                loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=['accuracy'])

  return model

In [41]:
# Instantiate the tuner and perform hypertuning
tuner = kt.Hyperband(model_builder,
                     objective='val_accuracy',
                     max_epochs=10,
                     factor=3,
                     directory='tensorflow_learning_dir',
                     project_name='intro_to_kt')

In [42]:
# Create a callback to stop training early after reaching a certain value for the validation loss.
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

In [43]:
tuner.search(img_train, label_train, epochs=50, validation_split=0.2, callbacks=[stop_early])

# Get the optimal hyperparameters
best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]

print(f"""
The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is {best_hps.get('units')} and the optimal learning rate for the optimizer
is {best_hps.get('learning_rate')}.
""")

Trial 30 Complete [00h 00m 47s]
val_accuracy: 0.8826666474342346

Best val_accuracy So Far: 0.887583315372467
Total elapsed time: 00h 09m 47s

The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is 352 and the optimal learning rate for the optimizer
is 0.001.



In [44]:
# Build the model with the optimal hyperparameters and train it on the data for 50 epochs
model = tuner.hypermodel.build(best_hps)
history = model.fit(img_train, label_train, epochs=50, validation_split=0.2)

val_acc_per_epoch = history.history['val_accuracy']
best_epoch = val_acc_per_epoch.index(max(val_acc_per_epoch)) + 1
print('Best epoch: %d' % (best_epoch,))

Epoch 1/50
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - accuracy: 0.7937 - loss: 0.5653 - val_accuracy: 0.8468 - val_loss: 0.4176
Epoch 2/50
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.8479 - loss: 0.4181 - val_accuracy: 0.8580 - val_loss: 0.3846
Epoch 3/50
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.8585 - loss: 0.3852 - val_accuracy: 0.8616 - val_loss: 0.3815
Epoch 4/50
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.8634 - loss: 0.3653 - val_accuracy: 0.8652 - val_loss: 0.3623
Epoch 5/50
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.8698 - loss: 0.3500 - val_accuracy: 0.8732 - val_loss: 0.3580
Epoch 6/50
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.8748 - loss: 0.3390 - val_accuracy: 0.8761 - val_loss: 0.3394
Epoch 7/50
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.8792 - loss: 0.3251 - val_accuracy: 0.8761 - val_loss: 0.3474
Epoch 8/50
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.8830 - loss: 0.3150 - 

In [45]:
# Re-instantiate the hypermodel and train it with the optimal number of epochs from above.
hypermodel = tuner.hypermodel.build(best_hps)

# Retrain the model
hypermodel.fit(img_train, label_train, epochs=best_epoch, validation_split=0.2)

Epoch 1/36
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.7986 - loss: 0.5562 - val_accuracy: 0.8393 - val_loss: 0.4408
Epoch 2/36
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.8467 - loss: 0.4223 - val_accuracy: 0.8615 - val_loss: 0.3793
Epoch 3/36
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.8577 - loss: 0.3854 - val_accuracy: 0.8551 - val_loss: 0.3953
Epoch 4/36
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.8661 - loss: 0.3630 - val_accuracy: 0.8761 - val_loss: 0.3467
Epoch 5/36
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.8701 - loss: 0.3503 - val_accuracy: 0.8810 - val_loss: 0.3376
Epoch 6/36
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.8748 - loss: 0.3371 - val_accuracy: 0.8771 - val_loss: 0.3296
Epoch 7/36
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.8798 - loss: 0.3228 - val_accuracy: 0.8794 - val_loss: 0.3262
Epoch 8/36
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.8814 - loss: 0.3172 - 

In [47]:
# To finish this tutorial, evaluate the hypermodel on the test data.
eval_result = hypermodel.evaluate(img_test, label_test)
print("[test loss, test accuracy]:", eval_result)

313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8908 - loss: 0.3499
[test loss, test accuracy]: [0.34989795088768005, 0.8907999992370605]
